# Day 2 - Document Q&A with RAG using Chroma

Welcome back to the Kaggle 5-day Generative AI course!

Two big limitations of LLMs are 1) that they only "know" the information that they were trained on, and 2) that they have limited input context windows. A way to address both of these limitations is to use a technique called Retrieval Augmented Generation, or RAG. A RAG system has three stages:

1. Indexing
2. Retrieval
3. Generation

Indexing happens ahead of time, and allows you to quickly look up relevant information at query-time. When a query comes in, you retrieve relevant documents, combine them with your instructions and the user's query, and have the LLM generate a tailored answer in natural language using the supplied information. This allows you to provide information that the model hasn't seen before, such as product-specific knowledge or live weather updates.

In this notebook you will use the Gemini API to create a vector database, retrieve answers to questions from the database and generate a final answer. You will use [Chroma](https://docs.trychroma.com/), an open-source vector database. With Chroma, you can store embeddings alongside metadata, embed documents and queries, and search your documents.

---

LLM の 2 つの大きな制限は、1) トレーニングを受けた情報しか「知らない」こと、2) 入力コンテキスト ウィンドウが限られていることです。これらの両方の制限に対処する方法は、Retrieval Augmented Generation、またはRAGと呼ばれる技術を使用することです。 RAGシステムには3つの段階があります。

インデックス作成

検索

生成

インデックス作成は事前に行われ、クエリ時に関連情報をすばやく検索できます。クエリが入ると、関連するドキュメントを取得し、それらをあなたの指示とユーザーのクエリと組み合わせて、LLMに提供された情報を使用して自然言語でカスタマイズされた回答を生成します。これにより、製品固有の知識やライブ天気の更新など、モデルがこれまでに見たことのない情報を提供できます。

このノートでは、Gemini APIを使用してベクトルデータベースを作成し、データベースから質問に対する回答を取得し、最終的な回答を生成します。オープンソースのベクターデータベースであるChromaを使用します。Chromaを使用すると、メタデータと一緒に埋め込みを保存したり、ドキュメントやクエリを埋め込んだり、ドキュメントを検索したりできます。

# Setup

In [3]:
!pip install -qU "google-genai" "chromadb"

from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

# Explore available models

You will be using the [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) API method to calculate embeddings in this guide. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about the embedding models on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding).

`text-embedding-004` is the most recent generally-available embedding model, so you will use it for this exercise, but try out the experimental `gemini-embedding-exp-03-07` model too.

---

このガイドでは、embedContent APIメソッドを使用して埋め込みを計算します。Models.listエンドポイントを介してそれをサポートするモデルを見つけます。埋め込みモデルの詳細については、モデルページでも確認できます。

Text-embedding-004は、一般的に利用可能な最新の埋め込みモデルであるため、この演習には使用しますが、実験的なgemini-embedding-exp-03-07モデルも試してください。

In [14]:
GOOGLE_GENAI_USE_VERTEXAI=True
GOOGLE_CLOUD_PROJECT='your-project-id' # #insert your-project-id in VERTEXAI
GOOGLE_CLOUD_LOCATION='us-central1'

In [39]:
client = genai.Client(vertexai = GOOGLE_GENAI_USE_VERTEXAI, project = GOOGLE_CLOUD_PROJECT, location = GOOGLE_CLOUD_LOCATION)

for m in client.models.list():
    print(m.name)

publishers/google/models/chat-bison
publishers/google/models/text-bison
publishers/google/models/imageclassification-efficientnet
publishers/google/models/occupancy-analytics
publishers/google/models/multimodalembedding
publishers/google/models/pt-test
publishers/google/models/imageclassification-vit
publishers/google/models/bert-base
publishers/google/models/vehicle-detector
publishers/google/models/language-v1-classify-text-v1
publishers/google/models/language-v1-analyze-sentiment
publishers/google/models/language-v1-analyze-entity-sentiment
publishers/google/models/language-v1-analyze-syntax
publishers/google/models/resnet50
publishers/google/models/resnet-rs-50
publishers/google/models/imagesegmentation-deeplabv3
publishers/google/models/imageobjectdetection-yolo
publishers/google/models/imageobjectdetection-spinenet
publishers/google/models/owlvit-base-patch32
publishers/google/models/object-detector
publishers/google/models/ppe-detector
publishers/google/models/people-blur
publis

# Data

Here is a small set of documents you will use to create an embedding database.

In [20]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

# Creating the embedding database with ChromaDB

Create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. In this task, you are implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, you will use `retrieval_query` for the *query* embeddings. Check out the [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) for the full list of supported tasks.

Key words: Documents are the items that are in the database. They are inserted first, and later retrieved. Queries are the textual search terms and can be simple keywords or textual descriptions of the desired documents.

---

カスタム関数を作成して、Gemini APIで埋め込みを生成します。このタスクでは、検索システムを実装しているため、ドキュメント埋め込みを生成するための task_type は retrieval_document です。後で、クエリの埋め込みにretarival_queryを使用します。サポートされているタスクの完全なリストについては、APIリファレンスを確認してください。

キーワード：ドキュメントは、データベースにあるアイテムです。それらは最初に挿入され、後で取得されます。クエリはテキスト検索用語であり、単純なキーワードまたは目的のドキュメントのテキスト説明にすることができます。

In [44]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True
    
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "RETRIEVAL_DOCUMENT"
        else:
            embedding_task = "RETRIEVAL_QUERY"
            
        response = client.models.embed_content(
            model = "text-embedding-005",
            contents = input,
            config = types.EmbedContentConfig(
                task_type = embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

---

次に、GeminiEmbeddingFunctionを使用するChromaデータベースクライアントを作成し、上記で定義したドキュメントをデータベースに入力します。

In [45]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name = DB_NAME, embedding_function = embed_fn)

db.add(documents = documents, ids = [str(i) for i in range(len(documents))])

/var/tmp/ipykernel_3676/1719463736.py:5: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embed_fn = GeminiEmbeddingFunction()


---
Confirm that the data was inserted by looking at the database.

データベースを見て、データが挿入されたことを確認します。

In [46]:
db.count()

3

In [49]:
# You can peek at the data too.
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[-4.03229967e-02,  1.41243562e-02,  2.56362278e-02,
         -2.36929134e-02, -4.38607670e-02,  6.20906912e-02,
          2.42945785e-03, -3.60977016e-02,  7.42293447e-02,
         -2.50318903e-03, -4.79878820e-02, -4.27644029e-02,
          4.64647487e-02, -7.22099394e-02,  1.11831827e-02,
          1.51985465e-02,  5.16759008e-02, -8.90041515e-02,
         -1.20806172e-02, -3.97515111e-02,  1.89136472e-02,
         -5.40359840e-02, -1.56769846e-02,  1.72826424e-02,
          4.86506755e-03, -3.84162627e-02,  8.17918777e-03,
         -8.01924057e-03, -3.18096429e-02, -2.64366642e-02,
          4.60791439e-02, -1.28534678e-02,  1.38517637e-02,
         -4.01852056e-02,  3.26985829e-02, -5.27528934e-02,
          5.29560223e-02,  5.80221564e-02,  4.23723861e-04,
          3.36390059e-03,  1.32459616e-02, -6.49850145e-02,
          1.75776631e-02, -4.25353609e-02, -5.61555400e-02,
          8.00736621e-03, -8.61104056e-02, -4.52163927e-02,
          2

# Retrieval: Find relevant documents

To search the Chroma database, call the query method. Note that you also switch to the retrieval_query mode of embedding generation.

--- 

Chromaデータベースを検索するには、クエリメソッドを呼び出します。また、埋め込み生成のretrieval_queryモードに切り替えることに注意してください。

In [53]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query =  "How do you use the touchscreen to play music?"

result = db.query(query_texts = [query], n_results = 1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

# Augmented generation: Answer the question

Now that you have found a relevant passage from the set of documents (the retrieval step), you can now assemble a generation prompt to have the Gemini API generate a final answer. Note that in this example only a single passage was retrieved. In practice, especially when the size of your underlying data is large, you will want to retrieve more than one result and let the Gemini model determine what passages are relevant in answering the question. For this reason it's OK if some retrieved passages are not directly related to the question - this generation step should ignore them.

---

ドキュメントのセット（検索ステップ）から関連するパッセージを見つけたので、生成プロンプトを組み立てて、Gemini APIに最終回答を生成させることができます。この例では、単一のパッセージのみが取得されたことに注意してください。実際には、特に基礎となるデータのサイズが大きい場合は、複数の結果を取得し、Geminiモデルに質問に答える際にどのパッセージが関連しているかを判断させる必要があります。このため、取得したパッセージの一部が質問に直接関係していなくても大丈夫です。この生成ステップはそれらを無視する必要があります。

In [59]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f""""You are a helpful and informative bot  that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"
    
print(prompt)

"You are a helpful and informative bot  that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



---
Now use the generate_content method to to generate an answer to the question.

In [60]:
answer = client.models.generate_content(
    model = "gemini-2.0-flash-001",
    contents = prompt
)

Markdown(answer.text)

If you want to listen to music, the first thing you need to do is find the "Music" icon on the touchscreen display in your Googlecar and simply touch it.


# NextStep

Congrats on building a Retrieval-Augmented Generation app!

To learn more about using embeddings in the Gemini API, check out the [Intro to embeddings](https://ai.google.dev/gemini-api/docs/embeddings) or to learn more fundamentals, study the [embeddings chapter](https://developers.google.com/machine-learning/crash-course/embeddings) of the Machine Learning Crash Course.

For a hosted RAG system, check out the [Semantic Retrieval service](https://ai.google.dev/gemini-api/docs/semantic_retrieval) in the Gemini API. You can implement question-answering on your own documents in a single request, or host a database for even faster responses.

---

Retrieval-Augmented Generationアプリの構築おめでとうございます！

ジェミニAPIでの埋め込みの使用について詳しくは、埋め込み入門を確認するか、基礎については、機械学習クラッシュコースの埋め込みの章を参照してください。

ホストされている RAG システムについては、Gemini API のセマンティック 検索サービスを確認してください。独自のドキュメントに質問への回答を単一のリクエストで実装したり、データベースをホストしてさらに高速な応答を実行したりできます。